# Laboratorio 7: Clasificación 🤗

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>


### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sivert Escaff 
- Nombre de alumno 2: Rodrigo Montecino 

### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/...../)

### Temas a tratar
- Clasificación en problemas desbalanceados
- Lightgbm y xgboost
- Pipelines

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.



### Objetivos principales del laboratorio
- Comprender cómo trabajar con problemas de clasificación con clases desbalanceadas.
- Aplicar los modelos lightgbm y xgboost.
- Practicar Pipelines

# Parte Teórica [12 puntos]




1. Explique cuál es la diferencia entre los datos de entrenamiento y validación. [1 punto]

2. Explique cuál es el principal desafío al trabajar problemas de clasificación con data no supervisada. [1 punto]

3. Explique en **sus palabras** qué es la matriz de confusión y para qué se utiliza. [1 puntos]

4. Escriba la fórmula de las siguientes métricas y explique con **sus palabras** cómo se interpretan. [1 punto cada uno]

  * Accuracy
  * Precision
  * Recall
  * F1 score

5. Explique qué métrica recomendaría para los siguientes contextos de clasificación. [1 punto cada uno]

  * Mantenimiento predictivo de fallas de maquinaria pesada en la industria minera.  
  * Detección de enfermedades altamente contagiosas.
  * Aprobación de créditos de alto riesgo.
  * Detección de crímenes.

6. Explique qué es la calibración de modelos y para qué se usa. [1 punto]


**Respuesta:**

*1 - Los datos de entrenamiento son los que se usan para que el modelo aprenda de ellos, es decir, le damos toda esa información para que se ajuste a los parámetros de funcionamiento, mientras que, los de validacion(test) son los que se utilizan para evaluar el rendimiento (aprendizaje) del mismo modelo con los datos.
2 - Es difícil poder evaluar que tan bueno es el modelo, es decir, posee dificultados dado que los datos suelen no estar etiquetados.
3 - Es una forma más visual de ver el rendimiento del modelo, en donde se agrupan los VP, VN, FN y FP, de esto es posible derivar las distintas métricas conocidas como Accuracy, Precisión, Recall y F1-score.
4 - 
- El Accuracy te da la proporción de predicciones correctas de todo el conjunto de predicciones


$\text{Accuracy} = \frac{VP + VN}{VP + VN + FP + FN}$

- La precision solamente te expresa la proporcion de predicciones totales positivas dieron del total, es decir, VP sobre FP con la suma de VP.


$\text{Precision} = \frac{VP}{VP + FP}$

- Recall se preocupa en considerar siempre el porcentaje de predicciones positivas, busca la proporción de verdaderos positivos que fueron predichos.

$\text{Recall} = \frac{VP}{VP + FN}$

- F1-score es una medida que toma la precisión y el recall y saca su media, se utiliza en los contextos cuando las clases a clasificar están muy desbalanceadas.

$F1 = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$


5 -
- Es importante no fallar y darle todo ese rango, por tanto utilizaría Recall
- Igual que la anterior, no se puede fallar, así que también usaría Recall, aunque f1 tambien seria una buena opcion ya que es bueno buscar no implementar recursos en personas que no tienen la enfermedad.
- Lo mas optimo se cree que seria el uso de la curva ROC y AUC, ya que la ventaja de usar AUC-ROC es que considera todos los posibles umbrales de decision, lo que ayuda a encontrar el equilibrio optimo para minimizar errores en la aprobación de creditos.
- Utilizaría Recall, pero igual sería importante mantener una raya, dado que, si por ejemplo se refiere a una bomba en un lugar público sería mucho alboroto si no lo hay realmente, por lo que buscaría un mejor punto, es decir, f1-score sería bien aplicado en este contexto.

6 - 
Con la calibracion de modelos nos aseguramos que las probabilidades que predice un modelo sean realmente confiables. Por ejemplo, si un modelo predice que algo tiene un 70% de probabilidad de ocurrir, tras calibrarlo, debería ocurrir cerca de 70 veces de cada 100. Esto nos ayuda a tomar decisiones más seguras basadas en estas predicciones.


In [2]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
#!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# UMAP
#!pip install umap-learn

# Parte práctica [48 puntos]

<p align="center">
  <img src="https://i.ibb.co/61L8z0w/renacin-by-volframio-dcirf4l-fullview.jpg"
" width="400">
</p>



Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones.

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con el csv `statsplayers.csv`, donde deberán aplicar algoritmos de aprendizaje supervisado de clasificación en base a características que describen de jugadores de fútbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [3]:
df_players = pd.read_csv('stats_players.csv')
df_players.head()

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11


## 1. Predicción de Seleccionados Nacionales [14 puntos]

<p align="center">
  <img src="https://www.futuro.cl/wp-content/uploads/2016/06/chile-argentina-meme-12.jpg" width="300">
</p>



### 1.1 Preprocesamiento [5 puntos]

Tareas:

1. Genere los labels para la clasificación binaria en una variable llamada `label`. Para esto, trabaje sobre el atributo `National_Position` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. [Sin puntaje]

2. Hecho esto, ¿cuántos se tienen ejemplos por cada clase? Comente lo que observa. [1 punto]

3. Genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guárdelo una variable llamada `col_transformer`. [2 puntos]

4. Comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material `Player_Stats_Report.html` que viene en el zip del lab), al igual que las transformaciones aplicadas. [2 puntos]

**Respuesta:**

In [4]:
df_players["Seleccionado"] = (~df_players["National_Position"].isna()).astype('int')
print(f'Cantidad de seleccionados: {df_players.Seleccionado.sum()}, no seleccionados: {len(df_players.Seleccionado)-df_players.Seleccionado.sum()}')

Cantidad de seleccionados: 1075, no seleccionados: 16513


In [5]:
df_players.columns

Index(['Name', 'Nationality', 'National_Position', 'Club_Position', 'Height',
       'Weight', 'Preffered_Foot', 'Age', 'Work_Rate', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys', 'Seleccionado'],
      dtype='object')

Las justificaciones del preprocesamiento a utilizar sobre cada columna son:
Se eliminaran las columnas de:
- Name: dado que el nombre no debería tener influencia en si es seleccionado o no.
- Nationality: dado para las selecciones se deberían considerar jugadores con solo una nacionalidad.
- National_Position: dado que fue utilizado para crear el label.

Se hará un OneHotEncoding a las columnas de:
- Club_position: dado que para las distintas posiciones son distintos stats los que importan, lo cual debe tomar en consideración el modelo.
- Preffered_Foot: dado que es importante saber que pie utiliza para patiar y relacionarlo con la posición.
- Work_Rate: dado que es importante poseer el ratio de trabajo que tiene. Se podría pasar a ordinal pero esta complejo dado que considera la relación de 2 ratios.

Se hara una estandarización sobre las siguientes columnas dado a que siguen una distribución ajustable a una normal:
- Height
- Weight
- Age
- Ball_Control
- Dribbling
- Marking 
- Sliding_Tackle
- Standing_Tackle
- Aggression
- Reactions
- Interceptions
- Vision
- Composure
- Crossing
- Short_Pass
- Long_Pass
- Acceleration
- Speed
- Stamina
- Strength
- Balance
- Agility
- Jumping
- Heading
- Shot_Power
- Finishing
- Long_Shots
- Curve
- Freekick_Accuracy
- Penalties
- Volleys'

Se hara una escalamiento de mínimo y máximo sobre las siguientes columnas dado que poseen sus límites bien definidos:
- Weak_foot
- Skill_Moves

In [6]:
preprocess_data = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'),  ['Club_Position', 'Preffered_Foot', 'Work_Rate']),
        ('StandardScaler', StandardScaler(),
         ['Height', 'Weight', 'Age', 
          'Ball_Control', 'Dribbling',
          'Marking', 'Sliding_Tackle', 'Standing_Tackle',
          'Aggression', 'Reactions', 'Interceptions', 'Vision',
          'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
          'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
          'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
          'Freekick_Accuracy', 'Penalties', 'Volleys']), 
        ('MinMaxScaler', MinMaxScaler(), ['Weak_foot', 'Skill_Moves'])])


### 1.2 Entrenamiento [3 puntos]

Ahora, vamos a entrenar los pipelines generados en los pasos anteriores. Para esto, debe realizar las siguientes tareas:

1. Separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio). En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`). [1 puntos]


2. Defina un pipeline llamado `pipeline_xgboost` y otro llamado `pipeline_lightgbm`. Estos pipelines deben tener el mismo ColumnTransformer definido en la sección de preprocesamiento, pero deben variar los clasificadores de acuerdo al nombre de cada pipeline. [1 puntos]

3. Entrene los pipelines. [1 punto]



**Respuesta:**

In [6]:
#pip install xgboost lightgbm

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/124.9 MB 558.5 kB/s eta 0:03:44
   ---------------------------------------- 0.4/124.9 MB 2.3 MB/s eta 0:00:54
   ---------------------------------------- 1.2/124.9 MB 6.1 MB/s eta 0:00:21
    --------------------------------------- 2.0/124.9 MB 8.0 MB/s eta 0:00:16
    --------------------------------------- 2.9/124.9 MB 9.9 MB/s eta 0:00:13
   - -------------------------------------- 4.0/124.9 MB 11.5 MB/s eta 0:00:11
   - -------------------------------------- 5.1/124.9 MB 13.0 MB/s eta 0:00:10
   - -------------------------------------- 6.1/124.9 MB 13.9 MB/s eta 0:00:09
   -- ----------

In [7]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Pipeline con XGBoost
pipeline_xgboost = Pipeline([
    ('Preprocessing', preprocess_data), 
    ('XGBoost', XGBClassifier())
])

# Pipeline con LightGBM
pipeline_lightgbm = Pipeline([
    ('Preprocessing', preprocess_data), 
    ('LightGBM', LGBMClassifier())
])


In [8]:
### Split train test
X_train, X_test, y_train, y_test = train_test_split(df_players.drop(columns = ['Name', 'Nationality', 'National_Position', 'Seleccionado']), 
                                                    df_players['Seleccionado'],
                                                    test_size=0.3, random_state=3,
                                                    stratify = df_players['Seleccionado'])

### Hacemos fit a los modelos 
pipeline_xgboost.fit(X_train, y_train)
pipeline_lightgbm.fit(X_train, y_train)

### Hacemos las predicciones de cada uno de los modelos
y_pred_pipeline_xgboost = pipeline_xgboost.predict(X_test)
y_pred_pipeline_lightgbm = pipeline_lightgbm.predict(X_test)

[LightGBM] [Info] Number of positive: 752, number of negative: 11559
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001295 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2528
[LightGBM] [Info] Number of data points in the train set: 12311, number of used features: 70
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.061084 -> initscore=-2.732483
[LightGBM] [Info] Start training from score -2.732483


### 1.3 Resultados [6 puntos]

1. Calcule las métricas accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional) para evaluar el rendimiento de los distintos modelos. Verifique sus resultados usando `classification_report`. [2 puntos]

2. Explique qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional) y cómo influye la cantidad de ejemplos por clase en los resultados obtenidos. [2 puntos]

3. Explique qué métrica le parece más adecuada y concluya qué modelo tiene un mejor desempeño. [2 puntos]

**Respuesta:**

In [9]:
for i, y_pred in enumerate([y_pred_pipeline_xgboost, y_pred_pipeline_lightgbm]):
    print(f'Reporte para el pipeline {i+1}')
    print(classification_report(y_test, y_pred, zero_division=0))
    print('------'*10)

Reporte para el pipeline 1
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      4954
           1       0.48      0.14      0.22       323

    accuracy                           0.94      5277
   macro avg       0.71      0.57      0.59      5277
weighted avg       0.92      0.94      0.92      5277

------------------------------------------------------------
Reporte para el pipeline 2
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      4954
           1       0.51      0.14      0.22       323

    accuracy                           0.94      5277
   macro avg       0.73      0.57      0.60      5277
weighted avg       0.92      0.94      0.92      5277

------------------------------------------------------------


## 2. Predicción de posiciones de jugadores [4 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="300">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas.

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características.

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo `Club_Position`, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF']
central_ataque = ['RW', 'CAM', 'LW']
central = ['RM', 'CM', 'LM']
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

La elección del clasificador se justificar en base a la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y se deben comentar los resultados obtenidos en la clasificación.

**Tareas:** [1 punto por tarea]

1. En un nuevo dataframe, aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección y guárdelos en la variable `label`.
2. Cuente cuántos por clase quedan.
3. Entrene el nuevo pipeline y ejecute una evaluación de este.  
4. Comente los resultados obtenidos.

**Respuesta:**

In [10]:
#Etiquetas
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']

def map_position(x):
    if x in ataque:
        return 'ataque'
    elif x in central_ataque:
        return 'central_ataque'
    elif x in central:
        return 'central'
    elif x in central_defensa:
        return 'central_defensa'
    elif x in defensa:
        return 'defensa'
    elif x in arquero:
        return 'arquero'

### Drop Sub y Res
df_players_filter = df_players[~df_players["Club_Position"].isin(['Sub','Res'])].copy()

### Mapeo posiciones
df_players_filter["Club_Position"] = df_players_filter["Club_Position"].apply(map_position)

### Eliminando NAN
df_players_filter=df_players_filter[~df_players_filter.Club_Position.isna()]

df_players_filter

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Seleccionado
0,Cristiano Ronaldo,Portugal,LS,central_ataque,185,80,Right,32,High / Low,4,...,95,85,92,93,90,81,76,85,88,1
1,Lionel Messi,Argentina,RW,central_ataque,170,72,Left,29,Medium / Medium,4,...,68,71,85,95,88,89,90,74,85,1
2,Neymar,Brazil,LW,central_ataque,174,68,Right,25,High / Medium,5,...,61,62,78,89,77,79,84,81,83,1
3,Luis Suárez,Uruguay,LS,ataque,182,85,Right,30,High / Medium,4,...,69,77,87,94,86,86,84,85,88,1
4,Manuel Neuer,Germany,GK,arquero,193,92,Right,31,Medium / Medium,4,...,78,25,25,13,16,14,11,47,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17298,Raymond Skjelde,Norway,NaN,defensa,185,75,Right,18,Medium / Medium,2,...,65,44,35,22,25,24,22,41,31,0
17301,Andrew O'Connor,Republic of Ireland,NaN,defensa,175,73,Left,21,Medium / Medium,3,...,55,43,26,28,29,30,30,31,20,0
17310,Tom Bolarinwa,England,NaN,central,180,80,Right,26,Medium / Medium,3,...,54,22,38,37,37,37,32,29,29,0
17312,Zak Mills,England,NaN,defensa,182,83,Right,24,Medium / Medium,4,...,61,43,22,24,28,30,32,39,31,0


In [11]:
for value in df_players_filter.Club_Position.unique():
    print(f'Cantidad de jugadores {value}: {(df_players_filter.Club_Position==value).sum()}')

Cantidad de jugadores central_ataque: 581
Cantidad de jugadores ataque: 430
Cantidad de jugadores arquero: 632
Cantidad de jugadores defensa: 1180
Cantidad de jugadores central: 907
Cantidad de jugadores central_defensa: 209


In [13]:
from sklearn import svm
### Split train test
X_train, X_test, y_train, y_test = train_test_split(df_players_filter.drop(columns = ['Name', 'Nationality', 'National_Position', 'Seleccionado', 'Club_Position']), 
                                                    df_players_filter['Club_Position'],
                                                    test_size=0.3, random_state=3,
                                                    stratify = df_players_filter['Club_Position'])

### Pipelines sin club position
preprocess_data = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'),  ['Preffered_Foot', 'Work_Rate']),
        ('StandardScaler', StandardScaler(),
         ['Height', 'Weight', 'Age', 
          'Ball_Control', 'Dribbling',
          'Marking', 'Sliding_Tackle', 'Standing_Tackle',
          'Aggression', 'Reactions', 'Interceptions', 'Vision',
          'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
          'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
          'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
          'Freekick_Accuracy', 'Penalties', 'Volleys']), 
        ('MinMaxScaler', MinMaxScaler(), ['Weak_foot', 'Skill_Moves'])])

#Creamos ahora los pipelines 
Pipe1 = Pipeline([('Preprocessing', preprocess_data),('SVC', svm.SVC())])

### Hacemos fit a los modelos 
Pipe1.fit(X_train, y_train)

### Hacemos las predicciones de cada uno de los modelos
y_pred_pipe1 = Pipe1.predict(X_test)

In [14]:
for i, y_pred in enumerate([y_pred_pipe1]):
    print(f'Reporte para el pipeline {i+1}')
    print(classification_report(y_test, y_pred, zero_division=0))
    print('------'*10)

Reporte para el pipeline 1
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.78      0.84      0.81       129
        central       0.58      0.67      0.62       272
 central_ataque       0.62      0.30      0.40       174
central_defensa       0.59      0.27      0.37        63
        defensa       0.80      0.96      0.87       354

       accuracy                           0.75      1182
      macro avg       0.73      0.67      0.68      1182
   weighted avg       0.74      0.75      0.73      1182

------------------------------------------------------------


## 3. Predicciones de Seleccionados Nacionales para el Jere Klein [30 puntos]

<center>
<img src='https://www.radioactiva.cl/wp-content/uploads/2024/04/Jere-Klein-1-768x432.webp' width=500 />

Después de alcanzar la fama como cantante urbano, Jere Klein decide explorar una nueva faceta. Con su amor por el fútbol y convencido de que los artistas urbanos poseen un talento y versatilidad excepcionales, Jere se embarca en un proyecto innovador: desarrollar un sistema de inteligencia artificial capaz de identificar a jugadores que tienen potencial para convertirse en futbolistas profesionales. Su teoría es que muchos artistas del género urbano chileno, con sus habilidades únicas y su disciplina, podrían destacarse también en el deporte. Con este sistema, Jere espera no solo abrir nuevas oportunidades para sus colegas artistas, sino también demostrar la amplia gama de talentos que pueden ofrecer.

### 2.1 ¿Qué modelo de árbol es más de "pana"? [10 puntos]

<center>
<img src='https://64.media.tumblr.com/39189215a7d3d96823cb359f35b44e05/tumblr_psmrhrR3Xw1qf5hjqo4_540.gif' width=300 />


**Tareas**


1. Considerando el la variable llamada `label` creada en la sección 1.1. Para determinar cuál modelo de árbol sería más adecuado para la tarea en cuestión, utilice PyCaret. Este deberá centrarse exclusivamente en modelos de tipo árbol. Jere ha especificado que busca un modelo que tome decisiones rápidamente y que tenga una baja tasa de falsos positivos, ya que planea invertir en estos jugadores. [3 puntos] 

Para la comparación, utilice los siguientes modelos:

```python
['et', 'rf', 'dt', 'xgboost', 'lightgbm', 'catboost']
```

2. Explique en brevemente que son los modelos de la siguiente lista `['et', 'rf', 'dt']` y como funcionan. [3 punto]

3. Tras realizar la comparación de modelos, seleccione aquel que muestre el mejor rendimiento en términos de velocidad y precisión, especialmente en la reducción de falsos positivos. Utilice la función `evaluate_model` de PyCaret para revisar y analizar los resultados obtenidos en los siguientes aspectos:

  - **Confusión Matrix**: ¿Cómo se encuentran la tasa de verdaderos positivos y verdaderos negativos?
  - **Threshold**: ¿Es acaso el umbral por defecto del modelo el mejor para las predicciones?
  - **Feature Importance**: ¿Cuáles son las variables con mejor desempeño? ¿A qué podría deberse esto?
  - **Learning Curve**: ¿El modelo presenta algún problema?

  [4 puntos]

**Respuesta**

In [ ]:
from pycaret.datasets import get_data
from pycaret.classification import *
import os

os.environ["PYCARET_CUSTOM_LOGGING_LEVEL"] = "CRITICAL"

#Continuar código aquí

### 2.2 Reducción de dimensionalidad [14 puntos]

<center>
<img src='https://i.kym-cdn.com/photos/images/original/002/258/560/668.gif' width=400 />

A pesar de los resultados obtenidos previamente, el manager de Jere ha solicitado el entrenamiento de un modelo de XGBoost utilizando los datos disponibles. Además, se debe proceder a realizar una reducción de dimensionalidad basada en la importancia de las características.

Para llevar a cabo esta tarea:

1. Inicie entrenando un modelo XGBoost con todas las características disponibles. [2 puntos]

2. Una vez el modelo esté entrenado, evalúe y clasifique las características según su importancia de forma descendente. [2 puntos]

3. Utilice esta clasificación para ejecutar una búsqueda recursiva de eliminación de características, eliminando progresivamente las menos importantes y evaluando el impacto en el desempeño del modelo hasta identificar las N características más críticas. [2 puntos]

4. Con este conjunto reducido de características, entrene un nuevo modelo y evalúe su rendimiento. [2 puntos]

5. Posteriormente, responda a las siguientes preguntas para una comprensión más profunda de los cambios y beneficios:

  - ¿El rendimiento del modelo con las características seleccionadas es similar al del modelo original? ¿Cómo se comparan en términos de precisión y robustez? [2 puntos]
  - ¿Cuáles son los beneficios potenciales de eliminar variables del modelo? Considere factores como la simplificación del modelo, reducción del tiempo de entrenamiento, y mejora en la capacidad de generalización. [2 puntos]
  - Comente si el modelo con menor dimensionalidad es más sencillo de explicar. Explique brevemente por qué la eliminación de ciertas características puede facilitar la comprensión y la explicación del comportamiento del modelo. [2 puntos]

Notar que con esta metodologia buscamos encontrar un punto entermedio entre número de festures y desempeño. por esto, si observa que al aumentar festires el aumento es despreciable, puede no considerar agregar más features a su modelo.

**Respuesta**

### 2.3 Calibración Probabilistica [6 puntos]

<center>
<img src='https://media2.giphy.com/media/l2Je4Ku0Cx292KWv6/200w.gif?cid=6c09b952y0sihtq9tb6sz8j2023x3zxxp3qx1ocgonkpkblj&ep=v1_gifs_search&rid=200w.gif&ct=g' width=400 />

Para lograr modelos más modulares, se recomienda realizar una calibración del modelo entrenado anteriormente, con el objetivo de obtener salidas que reflejen mayor modularidad.

1. Se solicita que utilice un método de calibración que asegure que las probabilidades generadas incrementen de manera monótona. Una métrica ampliamente utilizada para evaluar la precisión de la calibración de un modelo es el Brier Score. Calcule el Brier Score para el modelo tanto antes como después de la calibración. Esto le permitirá realizar una comparación cuantitativa y determinar si la calibración ha mejorado el rendimiento del modelo. Para más información sobre el Brier Score, puede consultar el siguiente enlace: [Scikit-Learn - Brier Score Loss](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.brier_score_loss.html). [3 puntos]

2. Tras la calibración, examine y comente los resultados obtenidos. A su análisis añada una comparación visual de las ideales versus las salidas del modelo original (sin calibrar) y del modelo calibrado. [3 puntos]

**Respuesta**